## **Q#** How can I calculate a score based on the materials? *(Jasmine)*

### Qualitative:
#### Problem -
- We have extracted the materials from 30 brands, how do we automatically calculate a score for the materials?
#### Hypothesis & Assumptions -
- 
#### Context, Motivation & Rationale -
- We want the Chrome Extension to access the links provided in the description of YouTube hauls on clothing, so that we can find the material and generate a sustainability score for the items featured in the video
#### Definitions, Data, and Methods -
- [Higg Materials Sustainability Index (Higg MSI)](https://howtohigg.org/higg-msi/an-introduction-to-msi/) - "material assessment tool that calculates the environmental impacts of materials used in consumer goods industries."
- Higg MSI measures environmental impact in 5 areas:
    - Global warming
    - Nutrient pollution in water (eutrophication)
    - Water scarcity
    - Abiotic resource depletion, use of fossil fuels
    - Chemistry
- [Material Score Calculator](https://www.selflessclothes.com/blog/sustainability-calculator/) - 
#### Biases & Assumptions
- 